---
layout: exercises
chapter: 7
chapter-title: Ensemble Learning and Random Forests
permalink: /ml-book/chapter7/exercises.html
---

## Exercise 1

If I've trained five different models on the exact same training data and they all achieve 95% precision there is a chance that I could combine these models to get better results. You would combine the 5 predictions and predict the majority vote and that could achieve a precision higher than 95%.

Accuracy: (TP + TN) / (FP + FN + TP + TN)
Precision: TP / (TP + FP)

## Exercise 2

A hard vote counts the weight of each vote equally when determining the majority vote, whereas a soft voting classifier will weight each vote according to it's `predict_proba` value.

## Exercise 3

Bagging is when you sample with replacement, so it would definitely be possible to distribute across multiple servers, you just need to copy all the data to each server. I think it would also be possible to distribute pasting ensembles across several servers, again just put a copy of the data on each server and sample without replacement. Boosting sounds difficult to spread across servers because each model needs to correct it's predecessor. Yes random forests are possible to distribute. Stacking would be partly possible since the parts before the blender could be distributed.

## Exercise 4

Out-of-bag evaluation allows you to gain an idea of how a bagging algorithm will perform on new data, because roughly 37% of data will not have been used in training the model. Therefore, this portion of the training sample can be used for evaluation much like a validation set.

## Exercise 5

Extra-Trees are more random than Random Forests because instead of choosing the optimal feature/threshold at each node (i.e. the feature (amongst a random subset) and threshold which best splits the data at the node), it randomly selects a threshold for each feature. Therefore, the resultant tree is less predictable since each split was made randomly based on these thresholds. They're faster than Random Forests since they don't require time to compute the optimal feature/threshold for each node.

## Exercise 6

If your AdaBoost ensemble underfits the training data you should increase the learning rate (defaults to 1). You can also try increasing the number of estimators or decreasing the default regularization parameters.

## Exercise 7

If your Gradient Boosting ensemble overfits then you should decrease the learning rate so that it will generalize better.